# Quick Start: Crystal Generation with GGen

This notebook shows the basics of generating and working with crystal structures using `GGen`.

**What you'll learn:**
- Generate a crystal structure from a chemical formula
- Optimize the structure using ML potentials (ORB)
- Inspect and export the structure


In [1]:
from ggen import GGen


Warp DeprecationWarning: The symbol `warp.vec` will soon be removed from the public API. Use `warp.types.vector` instead.


## Generate a Crystal Structure

Create a `GGen` instance and generate a structure. GGen will:
1. Find compatible space groups for your formula
2. Generate multiple candidate structures
3. Score them by energy and symmetry
4. Return the best one


In [ ]:
ggen = GGen()

result = ggen.generate_crystal(
    formula="BaTiO3",
    num_trials=10,           # Try 10 random structures (all get relaxed)
    optimize_geometry=True,  # Relax with ORB force field
    preserve_symmetry=True,  # Keep high symmetry during optimization
)

print(f"Formula: {result['name']}")
print(f"Space group: {result['final_space_group_symbol']} (#{result['final_space_group']})")
print(f"Energy: {result['best_crystal_energy']:.4f} eV")
print(f"Energy per atom: {result['best_crystal_energy'] / result['structure'].num_sites:.4f} eV/atom")


/Users/mmoderwell/.pyenv/versions/3.12.0/envs/materials/lib/python3.12/site-packages/orb_models/utils.py:30: UserWarning: Setting global torch default dtype to torch.float32.
  warnings.warn(f"Setting global torch default dtype to {torch_dtype}.")


Formula: BaTiO3 (Pm)
Space group: Pm (#6)
Energy: -40.0453 eV
Energy per atom: -8.0091 eV/atom


## Inspect the Structure

The generated structure is a pymatgen `Structure` object:


In [3]:
structure = result['structure']

print(f"Composition: {structure.composition}")
print(f"Number of atoms: {structure.num_sites}")
print(f"Volume: {structure.volume:.2f} Å³")
print(f"Density: {structure.density:.3f} g/cm³")
print(f"\nLattice parameters:")
print(f"  a={structure.lattice.a:.3f} Å, b={structure.lattice.b:.3f} Å, c={structure.lattice.c:.3f} Å")
print(f"  α={structure.lattice.alpha:.1f}°, β={structure.lattice.beta:.1f}°, γ={structure.lattice.gamma:.1f}°")


Composition: O3 Ti1 Ba1
Number of atoms: 5
Volume: 67.73 Å³
Density: 5.717 g/cm³

Lattice parameters:
  a=4.058 Å, b=4.183 Å, c=3.991 Å
  α=90.1°, β=90.1°, γ=89.7°


## Visualize the Structure

If you have `pymatviz` installed, you can visualize interactively:


In [4]:
try:
    from pymatviz import StructureWidget
    display(StructureWidget(structure))
except ImportError:
    print("Install pymatviz for interactive visualization: pip install pymatviz")
    print(structure)


## Export the Structure

Save to CIF format for use in other tools:


In [5]:
# Export as CIF
structure.to("BaTiO3.cif")
print("Saved to BaTiO3.cif")

# Or get CIF content directly from the generation result
cif_content = result['cif_content']
print(f"\nCIF preview:\n{cif_content[:500]}...")


Saved to BaTiO3.cif

CIF preview:
# generated using pymatgen
data_BaTiO3
_symmetry_space_group_name_H-M   Pm
_cell_length_a   4.05755214
_cell_length_b   3.99076444
_cell_length_c   4.18287303
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.29084320
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   6
_chemical_formula_structural   BaTiO3
_chemical_formula_sum   'Ba1 Ti1 O3'
_cell_volume   67.73128105
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
  2  'x,...


## Targeting Specific Space Groups

You can request a specific space group if you have a target structure in mind.
You are not guaranteed to get the exact space group you request, but GGen will make a best effort to generate a structure with the requested space group. Increasing num_trials will increase the likelihood of getting the target space group.

In [ ]:
# Generate perovskite BaTiO3 in cubic Pm-3m
result_cubic = ggen.generate_crystal(
    formula="BaTiO3",
    space_group=221,  # Pm-3m (cubic perovskite)
    num_trials=5,
    optimize_geometry=True,
)

print(f"Space group: {result_cubic['final_space_group_symbol']}")
print(f"Energy: {result_cubic['best_crystal_energy']:.4f} eV")


Space group: Pm-3m
Energy: -39.9848 eV


## Working with Complex Compositions

GGen handles multi-component systems automatically:


In [ ]:
# Generate a high-entropy alloy structure
result_hea = ggen.generate_crystal(
    formula="Fe4Co4Ni4Mn2Cr2",
    num_trials=15,
    optimize_geometry=True,
    preserve_symmetry=True,
)

print(f"Formula: {result_hea['name']}")
print(f"Space group: {result_hea['final_space_group_symbol']}")
print(f"Energy per atom: {result_hea['best_crystal_energy'] / 16:.4f} eV/atom")


Formula: Fe4Co4Ni4Mn2Cr2 (P4_2/mcm)
Space group: P4_2/mcm
Energy per atom: -7.6785 eV/atom


## Summary

In this notebook you learned how to:
- Initialize `GGen` and generate crystal structures
- Control generation with `num_trials`, `space_group`, and `optimize_geometry`
- Inspect structure properties (composition, lattice, density)
- Export structures to CIF format

**Next steps:**
- See `02_chemistry_exploration.ipynb` for systematic exploration of chemical systems
- See `03_structure_mutations.ipynb` for applying mutations and tracking trajectories
